<a href="https://colab.research.google.com/github/JhonatanCI/CheatSheets-SQL/blob/main/Diff_Triggers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En SQL, los triggers se pueden clasificar según el momento en que se ejecutan en relación a la operación que los activa. Los tres tipos principales de triggers son: BEFORE, AFTER e INSTEAD OF. A continuación, se describen las diferencias entre cada uno de ellos y se proporcionan ejemplos de su uso:

Trigger BEFORE:
Un trigger BEFORE se activa justo antes de que se ejecute la operación que lo ha desencadenado. Por lo tanto, se puede utilizar para modificar los datos que se van a insertar, actualizar o eliminar. El trigger puede modificar los valores de la fila o lanzar una excepción para abortar la operación.
Por ejemplo, suponga que queremos restringir el valor de un campo price en una tabla products para que no sea mayor a $100. Podemos utilizar un trigger BEFORE para asegurarnos de que cualquier operación de actualización o inserción no supere ese límite:

sql
Copy code
CREATE TRIGGER check_price
BEFORE INSERT OR UPDATE ON products
FOR EACH ROW
BEGIN
  IF NEW.price > 100 THEN
    SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Price should not be greater than 100';
  END IF;
END;
En este ejemplo, el trigger check_price se activa antes de cada operación de inserción o actualización en la tabla products. Si el valor del campo price de la nueva fila o la fila actualizada es mayor a 100, se lanza una excepción y la operación se aborta.

Trigger AFTER:
Un trigger AFTER se activa justo después de que se haya completado la operación que lo ha desencadenado. Por lo tanto, se puede utilizar para realizar tareas adicionales después de la operación.
Por ejemplo, suponga que queremos registrar cada vez que se actualiza el stock de un producto en la tabla inventory. Podemos utilizar un trigger AFTER para registrar la actualización en una tabla de auditoría:

In [ ]:
CREATE TRIGGER update_inventory
AFTER UPDATE ON inventory
FOR EACH ROW
BEGIN
  INSERT INTO inventory_audit (product_id, updated_at, quantity)
  VALUES (OLD.product_id, NOW(), NEW.stock - OLD.stock);
END;


En este ejemplo, el trigger update_inventory se activa después de cada operación de actualización en la tabla inventory. El código dentro del bloque BEGIN inserta una nueva fila en la tabla de auditoría inventory_audit, registrando el product_id actualizado, la fecha y hora actual y la cantidad actualizada.

Trigger INSTEAD OF:
Un trigger INSTEAD OF se utiliza para reemplazar la operación que lo ha desencadenado. Por lo tanto, se puede utilizar para realizar una acción diferente a la que se habría realizado normalmente. Esto se puede utilizar, por ejemplo, para permitir actualizaciones en una vista que no se corresponde con una única tabla subyacente.

Por ejemplo, suponga que tenemos una vista orders_summary que muestra un resumen de las órdenes realizadas en varias tablas de una base de datos. Queremos permitir que los usuarios actualicen los datos de esta vista, pero no queremos que se realicen actualizaciones directamente en las tablas subyacentes. Podemos utilizar un trigger INSTEAD OF para manejar estas actualizaciones:

In [ ]:
CREATE VIEW orders_summary AS
SELECT orders.order_id, customers.customer_name, order_details.quantity, products.product_name
FROM orders
JOIN customers ON orders.customer_id = customers.customer_id
JOIN order_details ON orders.order_id = order_details.order_id
JOIN products ON order_details.product_id = products.product_id;

CREATE TRIGGER orders_summary_update
INSTEAD OF UPDATE ON orders_summary
FOR EACH ROW
BEGIN
  UPDATE orders
  SET customer_id = (SELECT customer_id FROM customers WHERE customer_name = NEW.customer_name)
  WHERE order_id = NEW.order_id;
  
  UPDATE order_details
  SET product_id = (SELECT product_id FROM products WHERE product_name = NEW.product_name),
      quantity = NEW.quantity
  WHERE order_id = NEW.order_id;
END;

En este ejemplo, creamos una vista orders_summary que muestra información de varias tablas. Luego creamos un trigger INSTEAD OF llamado orders_summary_update que se activa cuando se intenta actualizar la vista. El código dentro del bloque BEGIN actualiza las tablas subyacentes orders y order_details en lugar de actualizar directamente la vista. La actualización se realiza utilizando los datos de la nueva fila actualizada (NEW) y las subconsultas para obtener los valores correspondientes de customer_id y product_id.

En resumen, los triggers BEFORE, AFTER e INSTEAD OF en SQL se diferencian en el momento en que se activan en relación a la operación que los desencadena. Cada tipo de trigger tiene un propósito específico y se puede utilizar para realizar diferentes tareas, como validar datos, registrar eventos o reemplazar una operación.